# Preprocessing
* For each Wikipedia text and Wikidata description collected:

    tokenise the text, lowercase the tokens, remove function words

* Store the results in a panda dataframe containing 5 columns:

    person, Wikipedia page text, Wikipedia page text after preprocessing, Wikidata description, Wikidata description after preprocessing

Note. To improve clustering and classification results, feel free to
add further pre-processing steps (eg Named entity recognition, postagging and extraction of e.g., nouns and verbs).

In [5]:
# tokenisation

In [2]:
# lower case tokens

In [3]:
# remove function words

In [4]:
# store in a dataframe